In [1]:
print("hello world")

hello world


In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=500)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki_tool.run("What is Machine Learning?"))

Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions. Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.
ML fi


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4


embeddings = OllamaEmbeddings(
    model="llama2",
)


loader=WebBaseLoader(web_path=("https://whc.unesco.org/en/list/252/"),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=( "rich-text", "tab_content", "tab_content_en")
                         )))

web_document=loader.load()


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,)
process_pdf = text_splitter.split_documents(web_document)
process_pdf[:10]

[Document(metadata={'source': 'https://whc.unesco.org/en/list/252/'}, page_content="An immense mausoleum of white marble, built in Agra between 1631 and 1648 by order of the Mughal emperor Shah Jahan in memory of his favourite wife, the Taj Mahal is the jewel of Muslim art in India and one of the universally admired masterpieces of the world's heritage.Immense mausolée funéraire de marbre blanc édifiée entre 1631 et 1648 à Agra sur l'ordre de l'empereur moghol Shah Jahan pour perpétuer le souvenir de son épouse favorite, le Taj Mahal, joyau le plus parfait de l'art musulman en Inde, est l'un des chefs-d'œuvre universellement admirés du patrimoine de l'humanité.إنّ تاج محل هو ضريح جنائزي هائل شُيّد من الرخام الأبيض بين عامي 1631 و1648 في أغرا بناءً على أوامر الإمبراطور المغولي شاه جهان بهدف تخليد ذكرى زوجته المفضّلة. ويشكّل تاج محل الذي يُعتبر أفضل جوهرة في الفن الإسلامي في الهند إحدى أبرز تُحف التراث البشري التي هي محطّ إعجاب العالم بأسره.泰姬陵是一座由白色大理石建成的巨大陵墓清真寺，是莫卧儿皇帝沙贾汗(Shah Jahan)为纪念

In [8]:
db=FAISS.from_documents(documents=process_pdf, embedding=embeddings)

In [9]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [11]:

from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever, name="UNESCO World Heritage Sites", description="Retrieves information about UNESCO World Heritage Sites.")

In [13]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=500)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

arxiv_tool.run("What is the latest research on federated learningin artificial intelligence?")


'Published: 2016-07-19\nTitle: Adaptive Artificial Intelligence in Games: Issues, Requirements, and a Solution through Behavlets-based General Player Modelling\nAuthors: Benjamin Ultan Cowley, Darryl Charles\nSummary: We present the last of a series of three academic essays which deal with the\nquestion of how and why to build a generalized player model. We propose that a\ngeneral player model needs parameters for subjective experience of play,\nincluding: player psychology, game structure, and actions'

In [14]:
tools=[
    wiki_tool,
    retriever_tool,
    arxiv_tool
]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\ProgramData\\Anaconda3\\envs\\rag\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='UNESCO World Heritage Sites', description='Retrieves information about UNESCO World Heritage Sites.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B15EADF740>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B1374808D0>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001B1CEEDDD00>, retriever=VectorStor

In [ ]:
### Agents

A big use case for LangChain is creating agents. Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action. After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish. This is often achieved via tool-calling.

In [16]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

C:\Users\shrey\AppData\Local\Temp\ipykernel_28380\2814759709.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")


In [19]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
prompt =ChatPromptTemplate.from_template("""
    Answer the following question based on the context below.
    Think stept by step and provide a final answer.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.   
    <context>
    {context}
    </context>
    Question: {input}""")

In [31]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,  # works with tools
    verbose=True
)

C:\Users\shrey\AppData\Local\Temp\ipykernel_28380\3467095508.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
response = agent.invoke({"input": "Tell me about the Taj Mahal."})
print(response)



> Entering new AgentExecutor chain...
Of course! The Taj Mahal is an iconic white marble mausoleum located in Agra, India. It was built by Mughal Emperor Shah Jahan in memory of his wife, Mumtaz Mahal, who died during childbirth in 1631. The construction of the Taj Mahal began in 1632 and took around 22 years to complete, with thousands of workers and artisans laboring to create this stunning monument.

The Taj Mahal is considered one of the most beautiful examples of Mughal architecture, which blends elements of Islamic, Persian, and Indian styles. The mausoleum features intricate inlay work of precious stones, including jasper, jade, and turquoise, as well as exquisite marble carvings. The Taj Mahal is also surrounded by beautiful gardens, fountains, and walkways, creating a serene and peaceful atmosphere.

The Taj Mahal has been designated as a UNESCO World Heritage Site since 1983 and is considered one of the Seven Wonders of the Modern World. It is visited by millions of people 